# Result Viewer

This notebook is used to visualise the results for a `Model`. Either a folder in `./results` can be specified by the user, or the notebook will find the latest results generated for `Model.config.model_name`. Dropdown menus allow results for a specific `Scenario` to be displayed.

**How to use:**

1. Specify a **model_results_directory** in the **Imports** cell, or leave as **None** to view the latest results for that model.
2. Run the **Imports**, **Model** and **Helper** cells first.
3. Use the dropdowns to choose a scenario or metric for each plot.

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable, Callable, Tuple, Dict, List

import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import os

from firm_ce.fast_methods import static_m
from firm_ce.system.parameters import ScenarioParameters_InstanceType
from firm_ce.system.components import Fleet_InstanceType

project_root = Path.cwd().parent
os.chdir(project_root)

%matplotlib inline

model_results_directory = None # Change this to the model directory name to view results for a specific model run. E.g., Path("results/Model_Example_20250101_000000")
config_directory = "inputs/config"
data_directory = "inputs/data"

### Generate Model instance
Run this cell after your environment is set up to create an instance of `Model` based on the inputs in `config_directory`. The `Model` instance contains data inputs for all scenarios, including the `Generator`, `Storage`, and `Line` instances associated with the results.

In [ ]:
from firm_ce.model import Model

model = Model(config_directory=config_directory, data_directory=data_directory, logging_flag=False) 
model.config.model_name, list(model.scenarios.keys()) 

### Helper functions

In [ ]:
RESULTS_ROOT = Path("./results")
CSV_HEADER_ROWS = 5
POWER_CAPACITY_UNIT = "[GW]"
ENERGY_CAPACITY_UNIT = "[GWh]"
POWER_UNIT = "[MW]"
ENERGY_UNIT = "[MWh]"
COLOURS1 = plt.get_cmap("tab20b")
COLOURS2 = plt.get_cmap("tab20c")

@dataclass(frozen=True)
class HeaderRows:
    """
    Standard form of 5-row CSV header in FIRM results files.
    
    Attributes:
    -------
    asset_names (List[str]): Ordinary names of the asset to which a column of data relates. Typically, a name of
        a Generator, Storage, Line, or Node object, though there are some additional special cases.
    object_types (List[str]): The type of object of the asset to which the column of data relates (e.g., "Generator",
        "Storage", "Major Line", "Minor Line", "Node").
    asset_ids (List[str]): Model-level ID of the asset to which the column of data relates.
    column_names (List[str]): Name of the data type for the asset stored in the column.
    column_units (List[str]): Units for the data stored in the column (e.g., "[GW]").
    """
    asset_names: List[str]
    object_types: List[str]
    asset_ids: List[str]
    column_names: List[str]
    column_units: List[str]

    
    def __init__(self, csv_path: str | Path):
        """
        Read the first five rows of a CSV file and convert to lists of strings.

        Parameters:
        -------
        csv_path (str | Path): Path to a CSV file containing modelling results.
        """
        header_rows = pd.read_csv(csv_path, header=None, nrows=CSV_HEADER_ROWS)
        object.__setattr__(self, "asset_names",  header_rows.iloc[0, :].astype(str).tolist())
        object.__setattr__(self, "object_types", header_rows.iloc[1, :].astype(str).tolist())
        object.__setattr__(self, "asset_ids",    header_rows.iloc[2, :].astype(str).tolist())
        object.__setattr__(self, "column_names", header_rows.iloc[3, :].astype(str).tolist())
        object.__setattr__(self, "column_units", header_rows.iloc[4, :].astype(str).tolist())
        return None

def find_latest_run_dir_for_model(model_name: str) -> Path:
    """
    Finds all result directories for the provided model name and returns the latest directory.

    Assumes the full name of a directory containing results from a model contains the model name as a prefix.

    Parameters:
    -------
    model_name (str): Name of the Model defined in the `config.csv` configuration file.

    Returns:
    -------
    Path: Path to the most recently modified Model result directory for the specified model name.
    """
    result_folder_prefix = f"{model_name}_"
    candidate_paths = [candidate_path for candidate_path in RESULTS_ROOT.glob(f"{result_folder_prefix}*") if candidate_path.is_dir()]
    if not candidate_paths:
        raise FileNotFoundError(f"No result directories starting with '{result_folder_prefix}' under {RESULTS_ROOT.resolve()}")
    
    return max(candidate_paths, key=lambda candidate_path: candidate_path.stat().st_mtime)

def load_results_csv(scenario_dir: Path, filename: str, row_labels: bool = True) -> pd.DataFrame:
    """
    Load a CSV for a scenario directory (within the model results folder) that has a 5-row header
    followed by a table. The first column of the table may contain row labels.

    Parameters:
    -------
    scenario_dir (Path): Directory for a specific scenario within the model results folder.
    filename (str): Name of the results CSV file, including the file extension.
    row_labels (bool): Whether the results CSV file has row labels or not.

    Returns:
    -------
    pd.DataFrame: Pandas DataFrame containing the data from the results CSV file. Row labels are
        used as the index, values of header rows are stored as attributes.
    """
    csv_path = scenario_dir / filename
    if not csv_path.exists():
        raise FileNotFoundError(f"{filename} not found at {csv_path}")

    header_rows = HeaderRows(csv_path)
    data_df = pd.read_csv(csv_path, header=None, skiprows=5)

    if row_labels:
        data_df.index = data_df.iloc[:, 0]
        data_df = data_df.drop(columns=data_df.columns[0])

    data_df.attrs["asset_names"]  = [str(x) if not pd.isna(x) else "" for x in header_rows.asset_names]
    data_df.attrs["object_types"]  = [str(x) if not pd.isna(x) else "" for x in header_rows.object_types]
    data_df.attrs["asset_ids"]    = [str(x) if not pd.isna(x) else "" for x in header_rows.asset_ids]
    data_df.attrs["column_names"] = [str(x) if not pd.isna(x) else "" for x in header_rows.column_names]
    data_df.attrs["column_units"]        = [str(x) if not pd.isna(x) else "" for x in header_rows.column_units]

    return data_df

def build_results_group_map(model: Model, scenario_name: str) -> Dict[str, str]:
    """
    Build a mapping {asset name -> results group} for generators, storages, major_lines, minor_lines, and nodes for 
    the given scenario. Generator objects use Generator.fuel.name for the type, while Storage and Line objects
    use the unit_type attribute as the type.

    Parameters:
    -------
    model (Model): A Model instance defined according to the configuration files.
    scenario_name (str): Directory for a specific scenario within the model results folder.

    Returns:
    -------
    Dict[str, str]: A mapping of asset names to result groups, allowing for aggregated summary results to be 
        plotted.
    """
    unit_map = {}
    scenario = model.scenarios[scenario_name]    
    
    for generator in getattr(scenario.fleet, "generators", {}).values():
        name = getattr(generator, "name")
        fuel = getattr(generator, "fuel", None)
        unit_map[str(name)] = fuel.name if fuel else "Unknown"

    for storage in getattr(scenario.fleet, "storages", {}).values():
        name = getattr(storage, "name")
        unit_map[str(name)] = getattr(storage, "unit_type", "Unknown")

    for line in getattr(scenario.network, "major_lines", {}).values():
        name = getattr(line, "name")
        unit_map[str(name)] = getattr(line, "unit_type", "Unknown")

    for line in getattr(scenario.network, "minor_lines", {}).values():
        name = getattr(line, "name")
        unit_map[str(name)] = getattr(line, "unit_type", "Unknown")

    for node in getattr(scenario.network, "nodes", {}).values():
        name = getattr(node, "name")
        unit_map[str(name)] = "Node"
    
    return unit_map

def build_energy_balance_map(model: Model, scenario_name: str) -> Dict[str, str]:
    """
    Build a mapping {asset name helper string -> energy balance group} for generators, storages, major_lines, and nodes for 
    the given scenario. 

    Notes:
    -------
    - Helper strings based upon the asset names are used as the key, allowing for assets to have multiple types of data
    mapped to separate energy balance groups.
    - Positive and negative values are mapped to separate groups. This allows for storage charging power to be plotted with 
    other load values and rooftop PV to be plotted with generation values.

    Parameters:
    -------
    model (Model): A Model instance defined according to the configuration files.
    scenario_name (str): Directory for a specific scenario within the model results folder.

    Returns:
    -------
    Dict[str, str]: A mapping of asset name helper strings to energy balance groups, allowing for aggregated summary results 
        to be plotted.
    """
    unit_map = {}
    scenario = model.scenarios[scenario_name]    
    
    for generator in getattr(scenario.fleet, "generators", {}).values():
        name = getattr(generator, "name")+'_Dispatch_Generation'
        fuel = getattr(generator, "fuel", None)
        unit_map[str(name)] = fuel.name+" Generation" if fuel else "Unknown Generation"

    for generator in getattr(scenario.fleet, "generators", {}).values():
        name = getattr(generator, "name")+'_Remaining Energy'
        fuel = getattr(generator, "fuel", None)
        unit_map[str(name)] = fuel.name+" Remaining Energy" if fuel else "Unknown Remaining Energy"

    for storage in getattr(scenario.fleet, "storages", {}).values():
        name = getattr(storage, "name")+'_Dispatch_Generation'
        unit_map[str(name)] = getattr(storage, "unit_type", "Unknown")+" Discharge"

    for storage in getattr(scenario.fleet, "storages", {}).values():
        name = getattr(storage, "name")+'_Dispatch_Load'
        unit_map[str(name)] = getattr(storage, "unit_type", "Unknown")+" Charge"

    for storage in getattr(scenario.fleet, "storages", {}).values():
        name = getattr(storage, "name")+'_Stored Energy'
        unit_map[str(name)] = getattr(storage, "unit_type", "Unknown")+" Stored Energy"

    for line in getattr(scenario.network, "major_lines", {}).values():
        name = getattr(line, "name")+"_Flow"
        unit_map[str(name)] = getattr(line, "name")+" Flows"

    for node in getattr(scenario.network, "nodes", {}).values():
        name = getattr(node, "name")+"_Demand_Generation"
        unit_map[str(name)] = "Rooftop Solar PV"

    for node in getattr(scenario.network, "nodes", {}).values():
        name = getattr(node, "name")+"_Demand_Load"
        unit_map[str(name)] = "Demand"

    for node in getattr(scenario.network, "nodes", {}).values():
        name = getattr(node, "name")+"_Spillage_Load"
        unit_map[str(name)] = "Spillage"

    for node in getattr(scenario.network, "nodes", {}).values():
        name = getattr(node, "name")+"_Deficit_Generation"
        unit_map[str(name)] = "Unserved Energy"

    return unit_map

def build_asset_name_map_from_attrs(attrs: dict) -> Dict[int, str]:
    """
    Build a map of dataframe column indices to corresponding asset names.

    Notes:
    -------
    - Asset names are stored as an attribute on the dataframe that was returned by load_results_csv
    function.
    - Column names on the dataframe returned by load_results_csv are a range of integers by default.
    This is because header=None when reading the CSV file, due to the multi-line header requiring
    special handling.

    Parameters:
    -------
    attrs (dict): Attributes of the Pandas dataframe returned by load_results_csv.

    Returns:
    -------
    Dict[int, str]: Map of dataframe column indices to corresponding asset names.
    """
    asset_names: List[str] = attrs.get("asset_names", [])
    return {i: name for i, name in enumerate(asset_names)}

def aggregate_row_by_results_group(
    row_series: pd.Series,
    asset_map: Dict[str, str],
    column_indices: Iterable[int],
    asset_names_by_col: Dict[int, str],
) -> pd.Series:
    """
    Aggregate row by results group across the provided integer columns.

    Parameters:
    -------
    row_series (pd.Series): The row to aggregate (e.g., 'Total Capacity'). Its index is integer column IDs.
    asset_map (Dict[str, str]): Mapping from asset name -> results group to allow for simplified aggregation in plots.
    column_indices (Iterable[int]): Which integer columns to include. Could be pre-filtered to only include IDs with a 
        particular unit (e.g., "[GW]" or "[GWh]") or column name (e.g., "Annual Spillage").
    asset_names_by_col (Dict[int, str]): Mapping from integer column -> asset name.

    Returns:
    -------
    pd.Series: Row aggregated by results group.
    """
    aggregated_row = {}
    for col in column_indices:
        asset_name = asset_names_by_col.get(col, "")
        results_group = asset_map.get(asset_name, "Unknown")
        value = row_series.get(col, np.nan)
        if pd.isna(value):
            continue
        aggregated_row[results_group] = aggregated_row.get(results_group, 0.0) + float(value)
    return pd.Series(aggregated_row)

def aggregate_2d_by_results_group(
    dataframe: pd.DataFrame,
    unit_map: Dict[str, str],
    column_indices: Iterable[int],
    asset_names_by_col: Dict[int, str],
) -> pd.DataFrame:
    """
    Aggregate table by results group across the provided integer columns.
    
    Parameters:
    -------
    dataframe (pd.DataFrame): The table to aggregate. Its index is integer column IDs.
    asset_map (Dict[str, str]): Mapping from asset name -> results group to allow for simplified aggregation in plots.
    column_indices (Iterable[int]): Which integer columns to include. Could be pre-filtered to only include IDs with a 
        particular unit (e.g., "[GW]" or "[GWh]") or column name (e.g., "Annual Spillage").
    asset_names_by_col (Dict[int, str]): Mapping from integer column -> asset name.

    Returns:
    -------
    pd.DataFrame: Table with columns aggregated by results group.
    """
    aggregate_dict = {}
    row_len = len(dataframe.index)

    for col in column_indices:
        asset_name = asset_names_by_col.get(col, "")
        results_group = unit_map.get(asset_name, "Unknown")

        col_vals = dataframe.get(col).to_numpy(dtype=np.float64)
        col_vals /= 1000.0

        if results_group not in aggregate_dict:
            aggregate_dict[results_group] = np.zeros(row_len, dtype=np.float64)
        aggregate_dict[results_group] += col_vals

    return pd.DataFrame(aggregate_dict, index=[idx for idx in dataframe.index])

def select_energy_balance_columns(
    object_types: List[str],
    column_names: List[str],
    column_units: List[str],
) -> Tuple[List[int], List[int], List[int], List[int], List[int]]:
    """
    Builds a list of column indicies for different types of energy balance data. Different rules are 
    required to find column indicies for different types of data.

    Notes:
    -------
    - Power deficits are grouped with generation data.
    - Operational demand columns are required in generation data, since negative values indicate
    rooftop PV feed-in.
    - Spillage is grouped with load data.

    Parameters:
    -------
    object_types (List[str]): The type of object of the asset to which the column of data relates (e.g., "Generator",
        "Storage", "Major Line", "Minor Line", "Node").
    column_names (List[str]): Name of the data type for the asset stored in the column.
    column_units (List[str]): Units for the data stored in the column (e.g., "[GW]").

    Returns:
    -------
    Tuple[List[int], List[int], List[int], List[int], List[int]]: Each tuple is a list of column indicies corresponding
        to a particular type of energy balance data, allowing this data to be grouped together.
    """
    gen_cols = [
        i
        for i, name in enumerate(column_names)
        if name in {"Demand", "Deficit", "Dispatch"}
    ]
    load_cols = [
        i
        for i, (typ, name, unit) in enumerate(zip(object_types, column_names, column_units))
        if (typ == "Storage" or name in {"Spillage", "Demand"}) and unit == POWER_UNIT
    ]
    storage_cols = [
        i
        for i, name in enumerate(column_names)
        if name=="Stored Energy"
    ]
    flexible_cols = [
        i
        for i, name in enumerate(column_names)
        if name=="Remaining Energy"
    ]
    line_cols = [
        i
        for i, name in enumerate(column_names)
        if name=="Flow"
    ]
    return gen_cols, load_cols, storage_cols, flexible_cols, line_cols

def aggregate_stream(
    csv_path: str | Path,
    first_t: int,
    nrows: int,
    usecols: List[int],
    group_names: List[str],
    chunksize: int = 4096,
) -> pd.DataFrame:
    """
    Stream-selected columns from a CSV, rename columns to `group_names`, sum duplicate group columns, and 
    return a single concatenated DataFrame.

    Allows large amounts of energy balance data to be loaded in chunks, minimising memory usage.

    Parameters:
    -------
    csv_path (str | Path): Path to a CSV file containing energy balance results. Typically, `energy_balance_ASSETS.csv`.
    first_t (int): Index of the first time interval in the period to be plotted. Typically, first interval of a year.
    nrows (int): Number of time intervals (i.e., rows of data) in the period to be plotted.
    usecols (List[int]): List of column indices to be read from the CSV.
    group_names (List[str]): List of the results group names across which columns are to be aggregated.
    chunksize (int): Number of rows to be read from CSV per chunk. Default of 4096 rows.

    Returns:
    -------
    pd.DataFrame: Table with columns aggregated by results group.
    """
    if not usecols:
        return pd.DataFrame()

    dtype = {c: "float32" for c in usecols} # float32 instead of float64 to reduce memory usage

    reader = pd.read_csv(
        csv_path,
        header=None,
        skiprows=CSV_HEADER_ROWS + first_t,
        nrows=nrows,
        usecols=usecols,
        dtype=dtype,
        chunksize=chunksize,
        engine="c",
    )

    output_chunks = []

    for chunk in reader:
        chunk_values = chunk.copy()
        chunk_values.columns = pd.Index(group_names)
        aggregated_values = chunk_values.T.groupby(level=0, sort=False).sum().T
        output_chunks.append(aggregated_values)

    if not output_chunks:
        return pd.DataFrame(columns=usecols)

    return pd.concat(output_chunks, ignore_index=True)

def invert_columns_by_mask(dataframe: pd.DataFrame, mask: pd.Series) -> None:
    """
    In-place multiply matching columns in the mask by -1.

    Parameters:
    -------
    dataframe (pd.DataFrame): A dataframe containing numeric values.
    mask (pd.Series): A boolean mask, where True values indicate columns to be multiplied by -1.

    Returns:
    -------
    None.

    Side-effects:
    -------
    The dataframe argument is modified in-place.
    """
    if dataframe is not None and not dataframe.empty:
        cols = dataframe.columns[mask.reindex(dataframe.columns, fill_value=False)]
        if len(cols):
            dataframe.loc[:, cols] *= -1

def stream_year_slice_aggregated(
    csv_path: str | Path,
    first_t: int,
    nrows: int,
    results_group_map: Dict[str, str],
    asset_names: List[str],
    object_types: List[str],
    column_names: List[str],
    column_units: List[str],
    chunksize: int = 4096,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    For a particular year, streams data from the energy balance results CSV file into a set of dataframes.
    Columns are aggregated in the dataframes according to simplified energy balance groups for generating plots.

    Parameters:
    -------
    csv_path (str | Path): Path to a CSV file containing energy balance results. Typically, `energy_balance_ASSETS.csv`.
    first_t (int): Index of the first time interval in the period to be plotted. Typically, first interval of a year.
    nrows (int): Number of time intervals (i.e., rows of data) in the period to be plotted.
    results_group_map: Dict[str, str]: A mapping of asset name helper strings to energy balance groups, allowing for 
        aggregated summary results to be plotted.
    asset_names: List[str]: Ordinary names of the asset to which a column of data relates. Typically, a name of
        a Generator, Storage, Line, or Node object, though there are some additional special cases.
    object_types (List[str]): The type of object of the asset to which the column of data relates (e.g., "Generator",
        "Storage", "Major Line", "Minor Line", "Node").
    column_names (List[str]): Name of the data type for the asset stored in the column.
    column_units (List[str]): Units for the data stored in the column (e.g., "[GW]").
    chunksize (int): Number of rows to be read from CSV per chunk. Default of 4096 rows.

    Returns:
    -------
    Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]: Each tuple corresponds to a set of energy
        balance data for plotting. Includes generation data, load data, stored energy data, flexible generator remaining 
        energy data, and line flows data.
    """
    gen_cols, load_cols, storage_cols, flexible_cols, line_cols = select_energy_balance_columns(object_types, column_names, column_units)

    if not gen_cols and not load_cols and not storage_cols and not flexible_cols and not line_cols:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    gen_keys  = [f"{asset_names[i]}_{column_names[i]}_Generation" for i in gen_cols]
    load_keys = [f"{asset_names[i]}_{column_names[i]}_Load"       for i in load_cols]
    storage_keys = [f"{asset_names[i]}_{column_names[i]}" for i in storage_cols]
    flexible_keys = [f"{asset_names[i]}_{column_names[i]}" for i in flexible_cols]
    line_keys = [f"{asset_names[i]}_{column_names[i]}" for i in line_cols]

    gen_group_names  = [results_group_map.get(k, "Unknown") for k in gen_keys]
    load_group_names = [results_group_map.get(k, "Unknown") for k in load_keys]
    storage_group_names = [results_group_map.get(k, "Unknown") for k in storage_keys]
    flexible_group_names = [results_group_map.get(k, "Unknown") for k in flexible_keys]
    line_group_names = [results_group_map.get(k, "Unknown") for k in line_keys]

    gen_df = aggregate_stream(
        csv_path=csv_path,
        first_t=first_t,
        nrows=nrows,
        usecols=gen_cols,
        group_names=gen_group_names,
        chunksize=chunksize,
    )
    # Invert Rooftop Solar PV in generation
    invert_columns_by_mask(gen_df, gen_df.columns.to_series().str.contains("Rooftop Solar PV"))

    load_df = aggregate_stream(
        csv_path=csv_path,
        first_t=first_t,
        nrows=nrows,
        usecols=load_cols,
        group_names=load_group_names,
        chunksize=chunksize,
    )
    # Invert all load groups except those containing "Demand"
    invert_columns_by_mask(load_df, ~load_df.columns.to_series().str.contains("Demand"))

    storage_df = aggregate_stream(
        csv_path=csv_path,
        first_t=first_t,
        nrows=nrows,
        usecols=storage_cols,
        group_names=storage_group_names,
        chunksize=chunksize,
    )

    flexible_df = aggregate_stream(
        csv_path=csv_path,
        first_t=first_t,
        nrows=nrows,
        usecols=flexible_cols,
        group_names=flexible_group_names,
        chunksize=chunksize,
    )

    line_df = aggregate_stream(
        csv_path=csv_path,
        first_t=first_t,
        nrows=nrows,
        usecols=line_cols,
        group_names=line_group_names,
        chunksize=chunksize,
    )

    return gen_df, load_df, storage_df, flexible_df, line_df

def set_slider_bounds(
    slider: widgets.IntRangeSlider,
    new_min: int,
    new_max: int,
    new_value: Tuple[int, int] | None = None,
    on_change_callable: Callable | None = None,
) -> None:
    """
    Safely set the bounds of the slider. Avoids errors where min>max or max<min when changing
    years and resetting the bounds.

    Parameters:
    -------
    slider (IntRangeSlider): An integer-range slider widget to control the x-axis range on the energy 
        balance plots.
    new_min (int): New minimum value of the slider.
    new_max (int): New maximum value of the slider.
    new_value (Tuple[int, int] | None = None): New value of the slider. For default None value, function
        will assign (new_min, new_max). 
    on_change_callable (Callable | None): A callable called when the slider range changes. Handles updating of 
        the slider in the interactive output.

    Returns:
    -------
    None.

    Side-effects:
    -------
    Changes the min and max attributes of the slider. Updates the slider according to the on_change_callable
        handler.
    """
    if on_change_callable is not None:
        slider.unobserve(on_change_callable, names="value")

    with slider.hold_trait_notifications():
        slider.min = int(new_min)
        slider.max = int(new_max)

        if new_value is None:
            new_value = (new_min, new_max)

        lo = max(new_min, min(int(new_value[0]), new_max))
        hi = min(new_max, max(int(new_value[1]), new_min))
        if lo > hi:
            lo, hi = new_min, new_max

        slider.value = (lo, hi)

    if on_change_callable is not None:
        slider.observe(on_change_callable, names="value")

def refresh_year_options_from_static(static: ScenarioParameters_InstanceType, year_dropdown: widgets.Dropdown, on_change_callable: Callable | None = None) -> None:
    """
    Update the options and value of the year dropdown when swapping to a different scenario.

    Parameters:
    -------
    static (ScenarioParameters_InstanceType): Represents the static parameters for a model scenario. Contains the
        start and end year that define the modelling horizon of the scenario.
    year_dropdown (widgets.Dropdown): A dropdown menu widget to set the calendar year of data to display.
    on_change_callable (Callable | None): A callable called when the slider range changes. Handles updating of 
        the slider in the interactive output.

    Returns:
    -------
    None.

    Side-effects
    -------
    year_dropdown.options and .value attributes may be modified.
    """
    years = list(range(static.first_year, static.final_year + 1))
    year_dropdown.unobserve(on_change_callable, names="value")
    
    if list(year_dropdown.options) != years: 
        year_dropdown.options = years
    desired_value = year_dropdown.value if year_dropdown.value in years else years[0]
    if year_dropdown.value != desired_value:
        year_dropdown.value = desired_value

    year_dropdown.observe(on_change_callable, names="value")
    return None

def refresh_asset_options_from_fleet(fleet: Fleet_InstanceType, asset_dropdown: widgets.Dropdown, on_change_callable: Callable | None = None) -> None:
    """
    Update the options and value of the asset dropdown when swapping to a different scenario.

    Parameters:
    -------
    fleet (Fleet_InstanceType): An instance of the Fleet jitclass for a given scenario. Contains the
        generator and storage objects for that scenario.
    asset_dropdown (widgets.Dropdown): A dropdown menu widget to set the asset data to display.
    on_change_callable (Callable | None): A callable called when the slider range changes. Handles updating of 
        the slider in the interactive output.

    Returns:
    -------
    None.

    Side-effects
    -------
    asset_dropdown.options and .value attributes may be modified.
    """
    gen_options = ["Generator_"+str(g.id)+"_"+g.name for g in fleet.generators.values()]
    storage_options = ["Storage_"+str(s.id)+"_"+s.name for s in fleet.storages.values()]
    asset_options = gen_options + storage_options
    asset_dropdown.unobserve(on_change_callable, names="value")

    if list(asset_dropdown.options) != asset_options:
        asset_dropdown.options = asset_options
    desired_value = asset_dropdown.value if asset_dropdown.value in asset_options else asset_options[0]
    if asset_dropdown.value != desired_value:
        asset_dropdown.value = desired_value

    asset_dropdown.observe(on_change_callable, names="value")
    return None

def slice_by_slider(dataframe: pd.DataFrame | None, slider: widgets.IntRangeSlider) -> pd.DataFrame:
    """
    Slice a dataframe to be bounded by the value of the x-axis slider widget.

    Parameters:
    -------
    dataframe (pd.DataFrame | None): The full dataframe to be sliced.
    slider (widgets.IntRangeSlider): An integer range slider to set the x-axis range for a timeseries plot.

    Returns:
    -------
    pd.DataFrame: The dataframe sliced to be within the bounds of the slider values (inclusive).
    """
    if dataframe is None or dataframe.empty:
        return pd.DataFrame()
    lo, hi = slider.value
    return dataframe.loc[(dataframe.index >= lo) & (dataframe.index <= hi)]

def build_asset_map(model: Model, scenario_name: str) -> Dict[str, str]:
    """
    Build a mapping {asset id -> asset id + asset name} for generators and storages for the given scenario. 

    Parameters:
    -------
    model (Model): A Model instance defined according to the configuration files.
    scenario_name (str): Directory for a specific scenario within the model results folder.

    Returns:
    -------
    Dict[str, str]: A mapping of asset ids to asset ids + asset names, allowing for time-series data for a 
    single asset to be filtered and plotted.
    """
    asset_map = {}
    scenario = model.scenarios[scenario_name]    
    
    for generator in getattr(scenario.fleet, "generators", {}).values():
        idx = getattr(generator, "id")
        name = getattr(generator, "name")
        asset_map["Generator_"+str(idx)+name+"_dispatch"] = "(Generator) "+str(idx)+" "+name

        if getattr(generator, "unit_type") == "flexible":
            asset_map["Generator_"+str(idx)+name+"_constraint"] = "(Generator) "+str(idx)+" "+name

    for storage in getattr(scenario.fleet, "storages", {}).values():
        idx = getattr(storage, "id")
        name = getattr(storage, "name")
        asset_map["Storage_"+str(idx)+name+"_dispatch"] = "(Storage) "+str(idx)+"_"+name
        asset_map["Storage_"+str(idx)+name+"_constraint"] = "(Storage) "+str(idx)+" "+name
    
    return asset_map

def stream_year_slice_asset(
    csv_path: str | Path,
    first_t: int,
    nrows: int,
    asset_map: Dict[str, str],
    asset_ids: List[str],
    asset_names: List[str],
    object_types: List[str],
    column_names: List[str],
    selected_asset: str,
    chunksize: int = 4096,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    For a particular year, streams data from the energy balance results CSV file into a pair of dataframes.

    Parameters:
    -------
    csv_path (str | Path): Path to a CSV file containing energy balance results. Typically, `energy_balance_ASSETS.csv`.
    first_t (int): Index of the first time interval in the period to be plotted. Typically, first interval of a year.
    nrows (int): Number of time intervals (i.e., rows of data) in the period to be plotted.
    asset_map: Dict[str, str]: A mapping of asset name helper strings to ordinary identifiers based on object types,
        asset ids and asset names.
    asset_ids: List[str]: Model-level ID of the asset to which the column of data relates.
    asset_names: List[str]: Ordinary names of the asset to which a column of data relates. Typically, a name of
        a Generator, Storage, Line, or Node object, though there are some additional special cases.
    object_types (List[str]): The type of object of the asset to which the column of data relates (e.g., "Generator",
        "Storage", "Major Line", "Minor Line", "Node").
    column_names (List[str]): Name of the data type for the asset stored in the column.
    selected_asset (str): Asset dropdown value for the selected asset.
    chunksize (int): Number of rows to be read from CSV per chunk. Default of 4096 rows.

    Returns:
    -------
    Tuple[pd.DataFrame, pd.DataFrame]: A pair of dataframes corresponding to dispatch data and energy constraint data
        for assets in the scenario.
    """
    base_key = selected_asset

    dispatch_cols = [
        i
        for i, name in enumerate(column_names)
        if name in {"Dispatch"}
        and f"{object_types[i]}_{asset_ids[i]}_{asset_names[i]}" == base_key
    ]
    constraint_cols = [
        i
        for i, name in enumerate(column_names)
        if name in {"Remaining Energy", "Stored Energy"}
        and f"{object_types[i]}_{asset_ids[i]}_{asset_names[i]}" == base_key
    ]

    if not dispatch_cols and not constraint_cols:
        return pd.DataFrame(), pd.DataFrame()

    dispatch_keys  = [f"{object_types[i]}_{asset_ids[i]}_{asset_names[i]}_dispatch" for i in dispatch_cols]
    constraint_keys = [f"{object_types[i]}_{asset_ids[i]}_{asset_names[i]}_constraint" for i in constraint_cols]

    dispatch_names  = [asset_map.get(k, "Unknown") for k in dispatch_keys]
    constraint_names = [asset_map.get(k, "Unknown") for k in constraint_keys]

    dispatch_dataframe = aggregate_stream(
        csv_path=csv_path,
        first_t=first_t,
        nrows=nrows,
        usecols=dispatch_cols,
        group_names=dispatch_names,
        chunksize=chunksize,
    )

    constraint_dataframe = aggregate_stream(
        csv_path=csv_path,
        first_t=first_t,
        nrows=nrows,
        usecols=constraint_cols,
        group_names=constraint_names,
        chunksize=chunksize,
    )   

    return dispatch_dataframe, constraint_dataframe



### Capacity Summary

In [ ]:
def make_capacity_plots(model_results_directory: Path | None):
    """
    Generate the power capacity and energy capacity plots for the model.

    Parameters:
    -------
    model_results_directory (Path | None): Path to the specified results directory for the Model. If None, then function
        will use the most recently modified results directory for the model.config.model_name.

    Side-effects:
    -------
    Generates widgets including a power capacity plot, energy capacity plot, dropdown menus for scenario and 
    data row selection, and some additional info/notes. Widgets are automatically updated when dropdown 
    selections are modified by the user.
    """
    if not model_results_directory:
        model_results_directory = find_latest_run_dir_for_model(model.config.model_name)

    # Initialise the widgets
    scenario_names = list(model.scenarios.keys())
    scenario_dropdown = widgets.Dropdown(
        options=scenario_names,
        description="Scenario:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px")
    )
    metric_dropdown = widgets.Dropdown(
        options=["Total Capacity", "New Build Capacity", "Min Build", "Max Build"],
        value="Total Capacity",
        description="Metric:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="250px")
    )
    info = widgets.HTML(value=f"<b>Results root:</b> {model_results_directory}")    

    power_capacity_plot = widgets.Output()
    energy_capacity_plot = widgets.Output()
    note = widgets.HTML(value="")

    def plot_series(series: pd.Series, title: str, y_label: str, x_label: str, message: str, output: widgets.Output) -> None:
        """
        Plot a series as a bar chart in the output context manager with the specified title and labels.

        Notes:
        -------
        - If series is empty or None, then a generic empty plot is passed to the output context manager
        with a message describing the error.

        Parameters:
        -------
        series (pd.Series): A series of aggregated data to be plotted in the bar chart. The index is the 
            results groups of the data.
        title (str): Title for the output figure.
        y_label (str): Label for the y-axis of the output figure.
        x_label (str): Label for the x-axis of the output figure.
        message (str): A short message describing any errors associated with creating the figure, such as missing
            results files.
        output (widgets.Output): The context manager within which the figure is contained.

        Side-effects:
        -------
        Creates and shows a plt.Figure in the output context manager widget.
        """
        with output:
            plt.figure(figsize=(10, 4))
            ax = plt.gca()
            if series is not None and not series.empty:
                colours=COLOURS1(np.linspace(0, 1, series.shape[0]))
                series.plot(kind="bar", ax=ax, color=colours)
                ax.set_title(title)
                ax.set_ylabel(y_label)
                ax.set_xlabel(x_label)
                ax.bar_label(ax.containers[0])
            else:
                ax.set_title(title)
                ax.set_ylabel(y_label)
                ax.set_xlabel(x_label)
                ax.set_xticks([])
                ax.text(0.5, 0.5, message, ha="center", va="center", transform=ax.transAxes)

            plt.tight_layout()
            plt.show()
    
    def update_plots(*args) -> None:
        """
        Callable that manages updates to the power capacity and energy capacity plots. Dropdown
        menu observers link to this callable to update plots when menu values change.

        Parameters:
        -------
        *args: Arbitrary arguments.

        Returns:
        -------
        None.

        Side-effects:
        -------
        Clears the output widget containing the plot, loads the data based upon the dropdown 
        menu values, aggregates data according to results groups and generates new plots in the 
        output. Updates the value of the note widget.
        """
        power_capacity_plot.clear_output(wait=True)
        energy_capacity_plot.clear_output(wait=True)
        
        scenario = scenario_dropdown.value
        scenario_directory = model_results_directory / f"{scenario}_full"

        msg = ""
        power_by_unit = pd.Series(dtype=float)
        energy_by_unit = pd.Series(dtype=float)

        if not scenario_directory.exists():
            msg = f"Folder not found: {scenario_directory.name}"
        elif not (scenario_directory / "capacities.csv").exists():
            msg = f"'capacities.csv' not found in {scenario_directory.name}"
        else:
            capacities_df = load_results_csv(scenario_directory, "capacities.csv")
            if metric_dropdown.value not in capacities_df.index:
                msg = f"Row '{metric_dropdown.value}' not found in capacities.csv"
            else:
                row = capacities_df.loc[metric_dropdown.value]
                results_group_map = build_results_group_map(model, scenario)

                units = capacities_df.attrs.get("column_units", [])
                power_capacity_columns = [i for i, u in enumerate(units) if u == "[GW]"]
                energy_capacity_columns = [i for i, u in enumerate(units) if u == "[GWh]"]

                asset_names_by_column = build_asset_name_map_from_attrs(capacities_df.attrs)

                power_by_unit = aggregate_row_by_results_group(row, results_group_map, power_capacity_columns, asset_names_by_column)
                energy_by_unit = aggregate_row_by_results_group(row, results_group_map, energy_capacity_columns, asset_names_by_column)    
        
        plot_series(
            power_by_unit,
            title=f"Power Capacity by Asset Type",
            y_label="Capacity [GW]",
            x_label="Asset Type",
            message=msg or "No data to display",
            output=power_capacity_plot
        )

        plot_series(
            energy_by_unit,
            title=f"Energy Capacity by Asset Type",
            y_label="Capacity [GWh]",
            x_label="Asset Type",
            message=msg or "No data to display",
            output=energy_capacity_plot
        )

        note.value = f"<i>{msg}</i>" if msg else ""
    

    # Set up event observation for dropdown values
    scenario_dropdown.observe(update_plots, names="value")
    metric_dropdown.observe(update_plots, names="value")
    
    # Display the widgets
    header = widgets.HBox([scenario_dropdown, metric_dropdown])
    display(info, header, note, widgets.HTML("<hr>"), power_capacity_plot, widgets.HTML("<hr>"), energy_capacity_plot)

    # Initialise the plots
    update_plots()

make_capacity_plots(model_results_directory)

### Cost Summary

In [ ]:
def make_cost_plots(model_results_directory: Path | None):
    """
    Generate the levelised cost and component cost plots for the model.

    Parameters:
    -------
    model_results_directory (Path | None): Path to the specified results directory for the Model. If None, then function
        will use the most recently modified results directory for the model.config.model_name.

    Side-effects:
    -------
    Generates widgets including a levelised cost plot, component cost plot, dropdown menu for scenario, and some 
    additional info/notes. Widgets are automatically updated when dropdown selections are modified by the user.
    """
    if not model_results_directory:
        model_results_directory = find_latest_run_dir_for_model(model.config.model_name)

    # Initialise the widgets
    scenario_names = list(model.scenarios.keys())
    scenario_dropdown = widgets.Dropdown(
        options=scenario_names,
        description="Scenario:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px")
    )
    info = widgets.HTML(value=f"<b>Results root:</b> {model_results_directory}")
    
    levelised_cost_plot = widgets.Output()
    component_cost_plot = widgets.Output()
    note = widgets.HTML(value="")

    def plot_stacked_dataframe(dataframe: pd.DataFrame, title: str, y_label: str, x_label: str, message: str, output: widgets.Output) -> None:
        """
        Plot a dataframe as a stacked bar chart in the output context manager with the specified title and labels.

        Notes:
        -------
        - If dataframe is empty or None, then a generic empty plot is passed to the output context manager
        with a message describing the error.

        Parameters:
        -------
        dataframe (pd.DataFrame): A dataframe of aggregated data to be plotted in the stacked bar chart. The column 
            names are the results groups of the data, the index is the row labels of the results CSV. Each index
            is plotted as a separate stacked bar of result groups.
        title (str): Title for the output figure.
        y_label (str): Label for the y-axis of the output figure.
        x_label (str): Label for the x-axis of the output figure.
        message (str): A short message describing any errors associated with creating the figure, such as missing
            results files.
        output (widgets.Output): The context manager within which the figure is contained.

        Side-effects:
        -------
        Creates and shows a plt.Figure in the output context manager widget.
        """
        with output:
            plt.figure(figsize=(10, 4))
            ax = plt.gca()
            if dataframe is not None and not dataframe.empty:
                colours=COLOURS1(np.linspace(0, 1, dataframe.shape[1]))
                dataframe.plot.bar(stacked=True, ax=ax, color=colours)
                ax.set_title(title)
                ax.set_ylabel(y_label)
                ax.set_xlabel(x_label)
                ax.legend(loc='right', bbox_to_anchor=(1.5, 0.5))
            else:
                ax.set_title(title)
                ax.set_ylabel(y_label)
                ax.set_xlabel(x_label)
                ax.set_xticks([])
                ax.text(0.5, 0.5, message, ha="center", va="center", transform=ax.transAxes)

            plt.tight_layout()
            plt.show()
    
    def update_plots(*args):
        """
        Callable that manages updates to the levelised cost and component cost plots. Dropdown
        menu observers link to this callable to update plots when menu values change.

        Parameters:
        -------
        *args: Arbitrary arguments.

        Returns:
        -------
        None.

        Side-effects:
        -------
        Clears the output widget containing the plot, loads the data based upon the dropdown 
        menu values, aggregates data according to results groups and generates new plots in the 
        output. Updates the value of the note widget.
        """
        levelised_cost_plot.clear_output(wait=True)
        component_cost_plot.clear_output(wait=True)
        
        scenario = scenario_dropdown.value
        scenario_directory = model_results_directory / f"{scenario}_full"

        msg = ""
        levelised_cost_values = pd.Series(dtype=float)
        cost_by_asset_type = pd.Series(dtype=float)

        if not scenario_directory.exists():
            msg = f"Folder not found: {scenario_directory.name}"
        elif not (scenario_directory / "levelised_costs.csv").exists():
            msg = f"'levelised_costs.csv' not found in {scenario_directory.name}"
        elif not (scenario_directory / "component_costs.csv").exists():
            msg = f"'component_costs.csv' not found in {scenario_directory.name}"
        else:
            levelised_cost_df = load_results_csv(scenario_directory, 'levelised_costs.csv')
            component_cost_df = load_results_csv(scenario_directory, 'component_costs.csv')
            
            asset_map = build_results_group_map(model, scenario)
            asset_names_by_column = build_asset_name_map_from_attrs(component_cost_df.attrs)

            cost_by_asset_type = aggregate_2d_by_results_group(component_cost_df, asset_map, component_cost_df.columns, asset_names_by_column)
            
            levelised_cost_values = levelised_cost_df.iloc[:, [1, 3, 4, 5]].copy()
            col_names = levelised_cost_df.attrs.get("column_names", [])
            levelised_cost_values.columns = [col_names[i] for i in [2, 4, 5, 6]]
        
        plot_stacked_dataframe(
            levelised_cost_values,
            title=f"Levelised Costs",
            y_label="Levelised Cost [$/MWh]",
            x_label="",
            message=msg or "No data to display",
            output=levelised_cost_plot
        )

        plot_stacked_dataframe(
            cost_by_asset_type,
            title=f"Component Costs by Asset Type",
            y_label="Cost ['000 $]",
            x_label="Cost Type",
            message=msg or "No data to display",
            output=component_cost_plot
        )

        note.value = f"<i>{msg}</i>" if msg else ""
    
    # Set up event observation for dropdown values
    scenario_dropdown.observe(update_plots, names="value")
    
    # Display the widgets
    header = widgets.HBox([scenario_dropdown])
    display(info, header, note, widgets.HTML("<hr>"), levelised_cost_plot, widgets.HTML("<hr>"), component_cost_plot)

    # Initialise the plots
    update_plots()

make_cost_plots(model_results_directory)

### Generation Summary

In [ ]:
def make_summary_plots(model_results_directory: Path | None):
    """
    Generate a set of summary energy plots for the model.

    Parameters:
    -------
    model_results_directory (Path | None): Path to the specified results directory for the Model. If None, then function
        will use the most recently modified results directory for the model.config.model_name.

    Side-effects:
    -------
    Generates widgets including a summary generation plot, summary storage discharge plot, summary demand plot, 
    summary spillage plot, summary unserved energy plot, summary transmission line flows plot, dropdown menu for 
    scenario, and some additional info/notes. Widgets are automatically updated when dropdown selections are modified
    by the user. Each plot consists of a stacked bar chart and a pie chart.
    """
    if not model_results_directory:
        model_results_directory = find_latest_run_dir_for_model(model.config.model_name)

    # Initialise the widgets
    scenario_names = list(model.scenarios.keys())
    scenario_dropdown = widgets.Dropdown(
        options=scenario_names,
        description="Scenario:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px")
    )
    info = widgets.HTML(value=f"<b>Results root:</b> {model_results_directory}")
    
    generation_plot = widgets.Output()
    discharge_plot = widgets.Output()
    demand_plot = widgets.Output()
    spillage_plot = widgets.Output()
    deficit_plot = widgets.Output()
    flows_plot = widgets.Output()
    note = widgets.HTML(value="")

    def plot_stacked_with_pie(dataframe_left: pd.DataFrame, pie_dataframe: pd.DataFrame, title_left: str, y_label_left: str, 
                              x_label_left: str, pie_title: str, message: str, output:widgets.Output):
        """
        Plot a dataframe as a stacked bar chart and a second single-row dataframe as a pie chart in the output context manager 
        with the specified titles and labels.

        Notes:
        -------
        - If a dataframe is empty or None, then a generic empty plot is passed to the output context manager
        with a message describing the error.

        Parameters:
        -------
        dataframe_left (pd.DataFrame): A dataframe of aggregated data to be plotted in the stacked bar chart. The column 
            names are the results groups of the data, the index is the row labels of the results CSV. Each index
            is plotted as a separate stacked bar of result groups.
        pie_dataframe (pd.DataFrame): A single-row dataframe to be plotted in a pie chart. The index provides the pie
            segments.
        title_left (str): Title for the stacked bar chart.
        y_label_left (str): Label for the y-axis of the stacked bar chart.
        x_label_left (str): Label for the x-axis of the stacked bar chart.
        pie_title (str): Title for the pie chart.
        message (str): A short message describing any errors associated with creating the figure, such as missing
            results files.
        output (widgets.Output): The context manager within which the figure is contained.

        Side-effects:
        -------
        Creates and shows a plt.Figure in the output context manager widget.
        """
        with output:
            _, (ax_bar, ax_pie) = plt.subplots(
                1, 2, figsize=(12, 4),
                gridspec_kw={'width_ratios': [2.2, 1]}
            )
            colours = None

            # Plot stacked bar chart on left of figure
            if dataframe_left is not None and not dataframe_left.empty:
                colours=COLOURS1(np.linspace(0, 1, dataframe_left.shape[1]))
                dataframe_left.plot.bar(stacked=True, ax=ax_bar, color=colours)
                ax_bar.set_title(title_left)
                ax_bar.set_ylabel(y_label_left)
                ax_bar.set_xlabel(x_label_left)
                legend = ax_bar.legend()
                legend.remove()
            else:
                ax_bar.set_title(title_left)
                ax_bar.set_ylabel(y_label_left)
                ax_bar.set_xlabel(x_label_left)
                ax_bar.set_xticks([])
                ax_bar.text(0.5, 0.5, message, ha="center", va="center", transform=ax_bar.transAxes)

            # Plot pie chart on right of figure
            if pie_dataframe is not None and isinstance(pie_dataframe, pd.DataFrame):
                if colours is None:
                    colours=COLOURS1(np.linspace(0, 1, pie_dataframe.shape[1]))
                values = pie_dataframe.iloc[0]

                if not values.empty:
                    wedges, _ = ax_pie.pie(
                        values.values,
                        startangle=90,
                        radius=1.2,
                        wedgeprops={"linewidth": 1, "edgecolor": "white"},
                        colors=colours
                    )

                    percentage_values = 100.0 * values.values / values.values.sum()
                    legend_labels = [f"{name} - {pct:.2f} %" for name, pct in zip(values.index, percentage_values)]

                    ax_pie.legend(
                        wedges, legend_labels,
                        loc="center left",
                        bbox_to_anchor=(1.05, 0.5),
                        fontsize=9,
                        frameon=False
                    ) 

                    ax_pie.set_ylabel("")        
                    ax_pie.set_title(pie_title)
                else:
                    ax_pie.set_title(pie_title)
                    ax_pie.text(0.5, 0.5, "No positive values", ha="center", va="center", transform=ax_pie.transAxes)
            else:
                ax_pie.set_title(pie_title)
                ax_pie.text(0.5, 0.5, message or "No data to display", ha="center", va="center", transform=ax_pie.transAxes)

            plt.tight_layout()
            plt.show()
    
    def update_plots(*args):
        """
        Callable that manages updates to the summary energy plots. Dropdown
        menu observers link to this callable to update plots when menu values change.

        Notes:
        -------
        - Lines and nodes are grouped by their asset name, rather than the results group.
        - Data for the pie chart is stored as a DataFrame as this is the return type of 
        aggregate_2d_by_results_group. It is converted to a Series in plot_stacked_with_pie. 

        Parameters:
        -------
        *args: Arbitrary arguments.

        Returns:
        -------
        None.

        Side-effects:
        -------
        Clears the output widget containing the plot, loads the data based upon the dropdown 
        menu values, aggregates data according to results groups and generates new plots in the 
        output. Updates the value of the note widget.
        """
        # Clear the context managers
        generation_plot.clear_output(wait=True)
        discharge_plot.clear_output(wait=True)
        demand_plot.clear_output(wait=True)
        spillage_plot.clear_output(wait=True)
        deficit_plot.clear_output(wait=True)
        flows_plot.clear_output(wait=True)
        
        # Set the scenario value
        scenario = scenario_dropdown.value
        scenario_directory = model_results_directory / f"{scenario}_full"

        # Create empty placeholder data
        msg = ""
        generation_by_results_group = pd.DataFrame(dtype=float)
        generation_by_results_group = pd.DataFrame(dtype=float)
        total_generation_by_results_group = pd.DataFrame(dtype=float)
        discharge_by_results_group = pd.DataFrame(dtype=float)
        total_discharge_by_results_group = pd.DataFrame(dtype=float)
        demand_by_node = pd.DataFrame(dtype=float)
        total_demand_by_node = pd.DataFrame(dtype=float)
        spillage_by_node = pd.DataFrame(dtype=float)
        total_spillage_by_node = pd.DataFrame(dtype=float)
        deficit_by_node = pd.DataFrame(dtype=float)
        total_deficit_by_node = pd.DataFrame(dtype=float)
        flow_by_line = pd.DataFrame(dtype=float)
        total_flow_by_line = pd.DataFrame(dtype=float)

        if not scenario_directory.exists():
            msg = f"Folder not found: {scenario_directory.name}"
        elif not (scenario_directory / "summary.csv").exists():
            msg = f"'summary.csv' not found in {scenario_directory.name}"
        else:
            summary_df = load_results_csv(scenario_directory, 'summary.csv')
            total_summary_series = summary_df.loc[["Total"]].copy()
            summary_df = summary_df.drop("Total")
            
            # Create maps from asset names to groups for plots
            results_group_map = build_results_group_map(model, scenario)
            asset_names_by_column = build_asset_name_map_from_attrs(summary_df.attrs)
            line_map = {line.name:line.name for line in model.scenarios[scenario].network.major_lines.values()}
            node_map = {node.name:node.name for node in model.scenarios[scenario].network.nodes.values()}

            # Select column IDs
            object_types = summary_df.attrs.get("object_types", [])
            node_col_types = summary_df.attrs.get("column_names", [])
            generator_columns = [i for i, u in enumerate(object_types) if u == "Generator"]
            storage_columns = [i for i, u in enumerate(object_types) if u == "Storage"]
            demand_columns = [i for i, u in enumerate(node_col_types) if u == "Annual Demand"]
            spillage_columns = [i for i, u in enumerate(node_col_types) if u == "Annual Spillage"]
            deficit_columns = [i for i, u in enumerate(node_col_types) if u == "Annual Deficit"]
            line_columns = [i for i, u in enumerate(object_types) if u == "Major Line"]

            # Aggregate data
            generation_by_results_group = aggregate_2d_by_results_group(summary_df, results_group_map, generator_columns, asset_names_by_column)
            total_generation_by_results_group = aggregate_2d_by_results_group(total_summary_series, results_group_map, generator_columns, asset_names_by_column)

            discharge_by_results_group = aggregate_2d_by_results_group(summary_df, results_group_map, storage_columns, asset_names_by_column)
            total_discharge_by_results_group = aggregate_2d_by_results_group(total_summary_series, results_group_map, storage_columns, asset_names_by_column)

            demand_by_node = aggregate_2d_by_results_group(summary_df, node_map, demand_columns, asset_names_by_column)
            total_demand_by_node = aggregate_2d_by_results_group(total_summary_series, node_map, demand_columns, asset_names_by_column)

            spillage_by_node = aggregate_2d_by_results_group(summary_df, node_map, spillage_columns, asset_names_by_column)
            total_spillage_by_node = aggregate_2d_by_results_group(total_summary_series, node_map, spillage_columns, asset_names_by_column)

            deficit_by_node = aggregate_2d_by_results_group(summary_df, node_map, deficit_columns, asset_names_by_column)
            total_deficit_by_node = aggregate_2d_by_results_group(total_summary_series, node_map, deficit_columns, asset_names_by_column)

            flow_by_line = aggregate_2d_by_results_group(summary_df, line_map, line_columns, asset_names_by_column)
            total_flow_by_line = aggregate_2d_by_results_group(total_summary_series, line_map, line_columns, asset_names_by_column)
        
        plot_stacked_with_pie(
            dataframe_left=generation_by_results_group,
            pie_dataframe=total_generation_by_results_group,
            title_left="Generation by Asset Type",
            y_label_left="Energy Generated [GWh]",
            x_label_left="Calendar Year",
            pie_title="Total Generation Mix",
            message=msg or "No data to display",
            output=generation_plot
        )
        plot_stacked_with_pie(
            dataframe_left=discharge_by_results_group,
            pie_dataframe=total_discharge_by_results_group,
            title_left="Discharge by Asset Type",
            y_label_left="Energy Discharged [GWh]",
            x_label_left="Calendar Year",
            pie_title="Total Storage Discharge Mix",
            message=msg or "No data to display",
            output=discharge_plot
        )
        plot_stacked_with_pie(
            dataframe_left=demand_by_node,
            pie_dataframe=total_demand_by_node,
            title_left="Operational Demand by Node",
            y_label_left="Net Energy Demand [GWh]",
            x_label_left="Calendar Year",
            pie_title="Total Net Demand Mix",
            message=msg or "No data to display",
            output=demand_plot
        )
        plot_stacked_with_pie(
            dataframe_left=-spillage_by_node,
            pie_dataframe=-total_spillage_by_node,
            title_left="Spillage by Node",
            y_label_left="Energy Spillage [GWh]",
            x_label_left="Calendar Year",
            pie_title="Total Spillage Mix",
            message=msg or "No data to display",
            output=spillage_plot
        )
        plot_stacked_with_pie(
            dataframe_left=deficit_by_node,
            pie_dataframe=total_deficit_by_node,
            title_left="Unserved Energy by Node",
            y_label_left="Unserved Energy [GWh]",
            x_label_left="Calendar Year",
            pie_title="Total Deficit Mix",
            message=msg or "No data to display",
            output=deficit_plot
        )
        plot_stacked_with_pie(
            dataframe_left=flow_by_line,
            pie_dataframe=total_flow_by_line,
            title_left="Flow Energy by Line",
            y_label_left="Flow Energy [GWh]",
            x_label_left="Calendar Year",
            pie_title="Total Flow Mix",
            message=msg or "No data to display",
            output=deficit_plot
        )

        note.value = f"<i>{msg}</i>" if msg else ""
    
    # Set up event observation for dropdown values
    scenario_dropdown.observe(update_plots, names="value")
    
    # Display the widgets
    header = widgets.HBox([scenario_dropdown])
    display(
        info, header, note, 
        widgets.HTML("<hr>"), generation_plot,
        widgets.HTML("<hr>"), discharge_plot,
        widgets.HTML("<hr>"), demand_plot,
        widgets.HTML("<hr>"), spillage_plot,
        widgets.HTML("<hr>"), deficit_plot,
        widgets.HTML("<hr>"), flows_plot,
    )

    # Initialise the plots
    update_plots()

make_summary_plots(model_results_directory)

### Energy Balance

In [ ]:
def make_energy_balance_plots(model_results_directory: Path | None):
    """
    Generate a set of interactive time-series plots for the model.

    Parameters:
    -------
    model_results_directory (Path | None): Path to the specified results directory for the Model. If None, then function
        will use the most recently modified results directory for the model.config.model_name.

    Side-effects:
    -------
    Generates widgets including an energy balance time-series plot, stored energy time-series plot, flexible generator 
    remaining energy time-series plot, transmission line flows time-series plot, x-axis range slider, dropdown menu for 
    scenario and year, and some additional info/notes. Widgets are automatically updated when dropdown selections or the slider
    values are modified by the user. Plots may take several seconds to update after interaction.
    """
    if not model_results_directory:
        model_results_directory = find_latest_run_dir_for_model(model.config.model_name)

    # Initialise the widgets
    scenario_names = list(model.scenarios.keys())
    scenario_dropdown = widgets.Dropdown(
        options=scenario_names,
        description="Scenario:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
    )
    year_dropdown = widgets.Dropdown(
        options=[],
        description="Year:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="250px"),
    )
    info = widgets.HTML(value=f"<b>Results root:</b> {model_results_directory}")
    energy_balance_plot = widgets.Output()
    storage_plot = widgets.Output()
    flexible_plot = widgets.Output()
    line_plot = widgets.Output()
    note = widgets.HTML(value="")

    slider = widgets.IntRangeSlider(
        value=[0, 8760],  
        min=0,
        max=8760,
        step=1,
        description="Range (days):",
        layout=widgets.Layout(width="800px"),
        continuous_update=False,
    )

    # Store the output states to manage update of widgets
    state = {
        "gen_aggregated": None,
        "load_aggregated": None,
        "storage_aggregated": None,
        "flexible_aggregated": None,
        "line_aggregated": None,
        "scenario": None,
        "year": None,
        "x_days": None,
        "updating": False,
    }

    def plot_stacked_areas_and_lines(area_dataframe: pd.DataFrame, line_dataframe: pd.DataFrame, title: str, 
                                     y_label: str, x_label: str, message: str, output: widgets.Output):
        """
        Plot a dataframe as a stacked bar chart in the output context manager with the specified title and labels.

        Notes:
        -------
        - If both dataframes are empty or None, then a generic empty plot is passed to the output context manager
        with a message describing the error.
        - If one dataframe is empty or None, then the other dataframe is still plotted.

        Parameters:
        -------
        area_dataframe (pd.DataFrame): A dataframe of aggregated data to be plotted in the stacked area chart. The column 
            names are the results groups of the data, the index is the time interval in days. Generation data on the 
            energy balance plot, stored energy on the storage plot, and remaining energy on the flexible generator plot.
        line_dataframe (pd.DataFrame): A dataframe of aggregated data to be plotted in the stacked line chart. The column 
            names are the results groups of the data, the index is the time interval in days. Load data on the energy balance
            plot.
        title (str): Title for the output figure.
        y_label (str): Label for the y-axis of the output figure.
        x_label (str): Label for the x-axis of the output figure.
        message (str): A short message describing any errors associated with creating the figure, such as missing
            results files.
        output (widgets.Output): The context manager within which the figure is contained.

        Side-effects:
        -------
        Creates and shows a plt.Figure in the output context manager widget.
        """
        output.clear_output(wait=True)
        with output:
            _, ax = plt.subplots(figsize=(20, 8))
            area_flag = area_dataframe is not None and not area_dataframe.empty
            line_flag = line_dataframe is not None and not line_dataframe.empty
            if area_flag or line_flag:
                if area_flag:
                    colours=COLOURS1(np.linspace(0, 1, area_dataframe.shape[1]))
                    area_dataframe.clip(lower=0).plot.area(stacked=True, ax=ax, lw=0, color=colours)
                if line_flag:
                    colours=COLOURS2(np.linspace(0, 1, line_dataframe.shape[1]))
                    line_dataframe.clip(lower=0).plot.line(stacked=True, ax=ax, color=colours)
                ax.set_title(title)
                ax.set_ylabel(y_label)
                ax.set_xlabel(x_label)
                ax.legend(loc="right", bbox_to_anchor=(1.15, 0.5))
            else:
                ax.set_title(title)
                ax.set_ylabel(y_label)
                ax.set_xlabel(x_label)
                ax.set_xticks([])
                ax.text(0.5, 0.5, message, ha="center", va="center", transform=ax.transAxes)
            plt.tight_layout()
            plt.show()

    def plot_line(line_dataframe: pd.DataFrame, title: str, y_label: str, x_label: str, message: str, output: widgets.Output):
        """
        Plot a dataframe as a line graph (not stacked) in the output context manager with the specified title and labels.

        Notes:
        -------
        - If dataframe is empty or None, then a generic empty plot is passed to the output context manager
        with a message describing the error.

        Parameters:
        -------
        line_dataframe (pd.DataFrame): A dataframe of aggregated data to be plotted in the line graph. The column 
            names are the results groups of the data, the index is the time interval in days. 
        title (str): Title for the output figure.
        y_label (str): Label for the y-axis of the output figure.
        x_label (str): Label for the x-axis of the output figure.
        message (str): A short message describing any errors associated with creating the figure, such as missing
            results files.
        output (widgets.Output): The context manager within which the figure is contained.

        Side-effects:
        -------
        Creates and shows a plt.Figure in the output context manager widget.
        """
        output.clear_output(wait=True)
        with output:
            _, ax = plt.subplots(figsize=(20, 8))
            if line_dataframe is not None and not line_dataframe.empty:
                colours=COLOURS1(np.linspace(0, 1, line_dataframe.shape[1]))
                line_dataframe.plot.line(stacked=False, ax=ax, color=colours)
                ax.set_title(title)
                ax.set_ylabel(y_label)
                ax.set_xlabel(x_label)
                ax.legend(loc="right", bbox_to_anchor=(1.15, 0.5))
            else:
                ax.set_title(title)
                ax.set_ylabel(y_label)
                ax.set_xlabel(x_label)
                ax.set_xticks([])
                ax.text(0.5, 0.5, message, ha="center", va="center", transform=ax.transAxes)
            plt.tight_layout()
            plt.show()

    def update_plots(*args):
        """
        Callable that manages updates to the time-series plots. Dropdown menu and slider observers 
        link to this callable to update plots when the values change.

        Notes:
        -------
        - May take about 15 seconds for plots to update upon interaction with a dropdown menu or slider.
        - Data is streamed in chunks from the results CSV file to minimise memory usage. Time-series data
        is memory intensive compared to the other results.
        - The `state` dictionary is used to track the state of some variables plotted on the figure, ensuring
        some values remain static during interactions while others are updated. 
        - The first time this function is called after an interaction, the state["updating"] is set to True.
        The updating state is then changed to False at the end of the function call. This prevents other
        observers modified during the state-change from re-calling the function.

        Parameters:
        -------
        *args: Arbitrary arguments.

        Returns:
        -------
        None.

        Side-effects:
        -------
        Clears the output widgets containing the plots, loads the data based upon the dropdown 
        menu and slider values, aggregates data according to results groups and generates new plots in the 
        output. Updates the value of the note widget.
        """
        # Block other observers from calling the function until the update is complete
        if state["updating"]:
            return
        state["updating"] = True
        
        # Clear existing outputs within the context managers
        energy_balance_plot.clear_output(wait=True)
        storage_plot.clear_output(wait=True)
        flexible_plot.clear_output(wait=True)
        line_plot.clear_output(wait=True)

        # Set the scenario
        scenario = scenario_dropdown.value
        scenario_directory = model_results_directory / f"{scenario}_full"
        static = model.scenarios[scenario].static
        resolution = float(static.resolution)

        # Initialise the output data
        msg = ""
        generation_by_group = pd.DataFrame()
        load_by_group = pd.DataFrame()
        storage_by_group = pd.DataFrame()
        flexible_by_group = pd.DataFrame()
        line_by_name = pd.DataFrame()

        if not scenario_directory.exists():
            msg = f"Folder not found: {scenario_directory.name}"
        elif not (scenario_directory / "energy_balance_ASSETS.csv").exists():
            msg = f"'energy_balance_ASSETS.csv' not found in {scenario_directory.name}"
        else:
            csv_path = scenario_directory / "energy_balance_ASSETS.csv"            

            # Update options/value in the year dropdown when the scenario changes
            if state["scenario"] != scenario:
                refresh_year_options_from_static(static, year_dropdown, update_plots)
                state["scenario"] = scenario

            # Set variables related to year selection
            selected_year = year_dropdown.value
            year_idx = selected_year - static.first_year
            first_t, last_t = static_m.get_year_t_boundaries(static, year_idx)
            nrows = last_t - first_t 

            # Stream data from `energy_balance_ASSETS.csv` and aggregate columns according to energy balance groups
            header = HeaderRows(csv_path)
            energy_balance_map = build_energy_balance_map(model, scenario)

            generation_by_group, load_by_group, storage_by_group, flexible_by_group, line_by_name = stream_year_slice_aggregated(
                csv_path=csv_path,
                first_t=first_t,
                nrows=nrows,
                results_group_map=energy_balance_map,
                asset_names=header.asset_names,
                object_types=header.object_types,
                column_names=header.column_names,
                column_units=header.column_units,
                chunksize=4096,
            )

            # Build x-axis in days
            x_hours = np.arange(first_t, last_t, dtype=float) * resolution
            x_days = x_hours / 24.0
            if not generation_by_group.empty:
                generation_by_group.index = x_days
            if not load_by_group.empty:
                load_by_group.index = x_days
            if not storage_by_group.empty:
                storage_by_group.index = x_days
            if not flexible_by_group.empty:
                flexible_by_group.index = x_days
            if not line_by_name.empty:
                line_by_name.index = x_days

            # Update data states
            state["gen_aggregated"] = generation_by_group
            state["load_aggregated"] = load_by_group
            state["storage_aggregated"] = storage_by_group
            state["flexible_aggregated"] = flexible_by_group
            state["line_aggregated"] = line_by_name
            state["x_days"] = x_days

            # Update slider to match the year span
            if state["year"] != selected_year or state["year"] is None:
                start_day = int(x_days[0]) if len(x_days) else 0
                end_day = int(x_days[-1]) if len(x_days) else 0
                set_slider_bounds(
                    slider,
                    start_day,
                    end_day,
                    on_change_callable=update_plots,
                )
                state["year"] = selected_year

        # Slice the aggregated data along the index (x-axis) according to slider values
        gen_df = slice_by_slider(state["gen_aggregated"], slider)
        load_df = slice_by_slider(state["load_aggregated"], slider)
        storage_df = slice_by_slider(state["storage_aggregated"], slider)
        flexible_df = slice_by_slider(state["flexible_aggregated"], slider)
        line_df = slice_by_slider(state["line_aggregated"], slider)

        # Plot the aggregated, sliced data
        plot_stacked_areas_and_lines(
            gen_df,
            load_df,
            title="Energy Balance",
            y_label="Power [MW]",
            x_label="Time [days]",
            message=msg or "No data to display",
            output=energy_balance_plot,
        )

        plot_stacked_areas_and_lines(
            storage_df,
            None,
            title="Energy Storage Systems",
            y_label="Stored Energy [MWh]",
            x_label="Time [days]",
            message=msg or "No data to display",
            output=storage_plot,
        )

        plot_stacked_areas_and_lines(
            flexible_df,
            None,
            title="Flexible Generator Constraints",
            y_label="Remaining Energy [MWh]",
            x_label="Time [days]",
            message=msg or "No data to display",
            output=flexible_plot,
        )

        plot_line(
            line_df,
            title="Line Flows",
            y_label="Flow [MW]",
            x_label="Time [days]",
            message=msg or "No data to display",
            output=line_plot,
        )
        note.value = f"<i>{msg}</i>" if msg else ""

        # Re-enable observers calling this function
        state["updating"] = False

    # Set up event observation for dropdown and slider values
    scenario_dropdown.observe(update_plots, names="value")
    year_dropdown.observe(update_plots, names="value")
    slider.observe(update_plots, names="value")

    # Display the widgets
    header = widgets.HBox([scenario_dropdown, year_dropdown])
    display(info, header, slider, note, widgets.HTML("<hr>"), energy_balance_plot, widgets.HTML("<hr>"), storage_plot, widgets.HTML("<hr>"), flexible_plot, widgets.HTML("<hr>"), line_plot)
    
    # Initialise the plots
    update_plots()

make_energy_balance_plots(model_results_directory)

### Asset Dispatch

In [ ]:
def make_asset_plots(model_results_directory: Path | None):
    """
    Generate a set of interactive time-series plots for the model.

    Parameters:
    -------
    model_results_directory (Path | None): Path to the specified results directory for the Model. If None, then function
        will use the most recently modified results directory for the model.config.model_name.

    Side-effects:
    -------
    Generates widgets including the asset time-series plot, x-axis range slider, dropdown menu for scenario and asset and year,
    and some additional info/notes. Widgets are automatically updated when dropdown selections or the slider
    values are modified by the user. Plots may take several seconds to update after interaction.
    """
    if not model_results_directory:
        model_results_directory = find_latest_run_dir_for_model(model.config.model_name)

    # Initialise the widgets
    scenario_names = list(model.scenarios.keys())
    scenario_dropdown = widgets.Dropdown(
        options=scenario_names,
        description="Scenario:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
    )
    year_dropdown = widgets.Dropdown(
        options=[],
        description="Year:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="250px"),
    )
    asset_dropdown = widgets.Dropdown(
        options=[],
        description="Asset Name",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="400px"),
    )
    info = widgets.HTML(value=f"<b>Results root:</b> {model_results_directory}")
    asset_plot = widgets.Output()
    note = widgets.HTML(value="")

    slider = widgets.IntRangeSlider(
        value=[0, 8760],  
        min=0,
        max=8760,
        step=1,
        description="Range (days):",
        layout=widgets.Layout(width="800px"),
        continuous_update=False,
    )

    # Store the output states to manage update of widgets
    state = {
        "asset_dispatch": None,
        "asset_constraint": None,
        "scenario": None,
        "year": None,
        "x_days": None,
        "updating": False,
    }

    def plot_dispatch_and_constraints(dispatch_dataframe: pd.DataFrame, constraint_dataframe: pd.DataFrame, title: str, 
                                     y_label_dispatch: str, y_label_constraint: str, x_label: str, message: str, output: widgets.Output):
        """
        Plot a dataframe as a stacked bar chart in the output context manager with the specified title and labels.

        Notes:
        -------
        - If both dataframes are empty or None, then a generic empty plot is passed to the output context manager
        with a message describing the error.
        - If one dataframe is empty or None, then the other dataframe is still plotted.

        Parameters:
        -------
        dispatch_dataframe (pd.DataFrame): A single-row dataframe of dispatch data for one asset to be plotted against the 
            primary y-axis. The column names are the results groups of the data, the index is the time interval in days.
        constraint_dataframe (pd.DataFrame): A single-row dataframe of constraint data for one asset to be plotted against the 
            scondary y-axis. The column names are the results groups of the data, the index is the time interval in days. 
            The constraint for storage systems is stored energy and for flexible generators is remaining energy.
        title (str): Title for the output figure.
        y_label_dispatch (str): Label for the primary y-axis of the output figure.
        y_label_constraint (str): Label for the secondary y-axis of the output figure.
        x_label (str): Label for the x-axis of the output figure.
        message (str): A short message describing any errors associated with creating the figure, such as missing
            results files.
        output (widgets.Output): The context manager within which the figure is contained.

        Side-effects:
        -------
        Creates and shows a plt.Figure in the output context manager widget.
        """
        output.clear_output(wait=True)
        with output:
            _, ax1 = plt.subplots(figsize=(20, 8))
            dispatch_flag = dispatch_dataframe is not None and not dispatch_dataframe.empty
            constraint_flag = constraint_dataframe is not None and not constraint_dataframe.empty
            if dispatch_flag or constraint_flag:
                if dispatch_flag:
                    dispatch_dataframe.columns = ["Dispatch"]
                    dispatch_dataframe.plot.line(ax=ax1, legend=False, color=COLOURS1(0))
                if constraint_flag:
                    ax2 = ax1.twinx()
                    constraint_dataframe.columns = ["Remaining Energy"]
                    constraint_dataframe.plot.line(ax=ax2, legend=False, color=COLOURS2(4))
                ax1.set_title(title)
                ax1.set_ylabel(y_label_dispatch)
                ax2.set_ylabel(y_label_constraint)
                ax1.set_xlabel(x_label)
                
                handles, labels = ax1.get_legend_handles_labels()
                if constraint_flag:
                    handles2, labels2 = ax2.get_legend_handles_labels()
                    handles += handles2
                    labels += labels2
                ax1.legend(handles, labels, loc="right", bbox_to_anchor=(1.15, 0.5))
            else:
                ax1.set_title(title)
                ax1.set_ylabel(y_label_dispatch)
                ax1.set_xlabel(x_label)
                ax1.set_xticks([])
                ax1.text(0.5, 0.5, message, ha="center", va="center", transform=ax1.transAxes)
            plt.tight_layout()
            plt.show()

    def update_plots(*args):
        """
        Callable that manages updates to the asset time-series plot. Dropdown menu and slider observers 
        link to this callable to update plots when the values change.

        Notes:
        -------
        - Data is streamed in chunks from the results CSV file to minimise memory usage. Time-series data
        is memory intensive compared to the other results.
        - The `state` dictionary is used to track the state of some variables plotted on the figure, ensuring
        some values remain static during interactions while others are updated. 
        - The first time this function is called after an interaction, the state["updating"] is set to True.
        The updating state is then changed to False at the end of the function call. This prevents other
        observers modified during the state-change from re-calling the function.

        Parameters:
        -------
        *args: Arbitrary arguments.

        Returns:
        -------
        None.

        Side-effects:
        -------
        Clears the output widgets containing the plot, loads the data based upon the dropdown 
        menu and slider values, aggregates data according to results groups and generates new plots in the 
        output. Updates the value of the note widget.
        """
        # Block other observers from calling the function until the update is complete
        if state["updating"]:
            return
        state["updating"] = True
        
        # Clear existing outputs within the context managers
        asset_plot.clear_output(wait=True)

        # Set the scenario
        scenario = scenario_dropdown.value
        scenario_directory = model_results_directory / f"{scenario}_full"
        static = model.scenarios[scenario].static
        fleet = model.scenarios[scenario].fleet
        resolution = float(static.resolution)

        # Initialise the output data
        msg = ""
        dispatch_by_name = pd.DataFrame()
        constraint_by_name = pd.DataFrame()

        if not scenario_directory.exists():
            msg = f"Folder not found: {scenario_directory.name}"
        elif not (scenario_directory / "energy_balance_ASSETS.csv").exists():
            msg = f"'energy_balance_ASSETS.csv' not found in {scenario_directory.name}"
        else:
            csv_path = scenario_directory / "energy_balance_ASSETS.csv"            

            # Update options/value in the year dropdown when the scenario changes
            if state["scenario"] != scenario:
                refresh_year_options_from_static(static, year_dropdown, update_plots)
                refresh_asset_options_from_fleet(fleet, asset_dropdown, update_plots)
                state["scenario"] = scenario

            # Set variables related to year and asset selection
            selected_year = year_dropdown.value
            year_idx = selected_year - static.first_year
            first_t, last_t = static_m.get_year_t_boundaries(static, year_idx)
            nrows = last_t - first_t 
            selected_asset = asset_dropdown.value

            # Stream data from `energy_balance_ASSETS.csv` and filter columns according to asset name and ID
            header = HeaderRows(csv_path)
            asset_map = build_asset_map(model, scenario)

            dispatch_by_name, constraint_by_name = stream_year_slice_asset(
                csv_path=csv_path,
                first_t=first_t,
                nrows=nrows,
                asset_map=asset_map,
                asset_ids=header.asset_ids,
                asset_names=header.asset_names,
                object_types=header.object_types,
                column_names=header.column_names,
                selected_asset=selected_asset,
                chunksize=4096,
            )

            # Build x-axis in days
            x_hours = np.arange(first_t, last_t, dtype=float) * resolution
            x_days = x_hours / 24.0
            if not dispatch_by_name.empty:
                dispatch_by_name.index = x_days
            if not constraint_by_name.empty:
                constraint_by_name.index = x_days

            # Update data states
            state["asset_dispatch"] = dispatch_by_name
            state["asset_constraint"] = constraint_by_name
            state["x_days"] = x_days

            # Update slider to match the year span
            if state["year"] != selected_year or state["year"] is None:
                start_day = int(x_days[0]) if len(x_days) else 0
                end_day = int(x_days[-1]) if len(x_days) else 0
                set_slider_bounds(
                    slider,
                    start_day,
                    end_day,
                    on_change_callable=update_plots,
                )
                state["year"] = selected_year

        # Slice the aggregated data along the index (x-axis) according to slider values
        dispatch_dataframe = slice_by_slider(state["asset_dispatch"], slider)
        constraint_dataframe = slice_by_slider(state["asset_constraint"], slider)

        # Plot the aggregated, sliced data
        plot_dispatch_and_constraints(
            dispatch_dataframe,
            constraint_dataframe,
            title="Asset Time-series",
            y_label_dispatch="Dispatch Power [MW]",
            y_label_constraint="Energy Constraint [MWh]",
            x_label="Time [days]",
            message=msg or "No data to display",
            output=asset_plot,
        )
        note.value = f"<i>{msg}</i>" if msg else ""

        # Re-enable observers calling this function
        state["updating"] = False

    # Set up event observation for dropdown and slider values
    scenario_dropdown.observe(update_plots, names="value")
    year_dropdown.observe(update_plots, names="value")
    asset_dropdown.observe(update_plots, names="value")
    slider.observe(update_plots, names="value")

    # Display the widgets
    header = widgets.HBox([scenario_dropdown, year_dropdown, asset_dropdown])
    display(info, header, slider, note, widgets.HTML("<hr>"), asset_plot)
    
    # Initialise the plots
    update_plots()

make_asset_plots(model_results_directory)